In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#%matplotlib notebook
import pandas as pd

from scipy.optimize import curve_fit as cf

Model_pre_trained = False

In [ ]:
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

In [ ]:
df = pd.read_csv("train.csv")
df_store = pd.read_csv("store.csv", index_col= 0)

## Model DateToDate

Basic model, it uses the sales of the pervious year to make it's prediction.

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
class DateToDateRegressor(BaseEstimator, RegressorMixin):
  '''
    The model takes a date frame composed of X = [store, day, month, year]
  '''
  def __init__(self, demo_param='demo'):
    self.demo_param = demo_param
  
  def fit(self, X, y):
    # Check that X and y have correct shape
    X, y = check_X_y(X, y)
    # Store the classes seen during fit
    self.year_ =  {}
    self.store_ = {}
    for i in range(len(X)): 
      if (X[i,0] in self.store_.keys()):
        if (X[i,2] in self.store_ [X[i,0]].keys()): 
          if ((X[i,1] in self.year_ [X[i,0]][X[i,2]].keys())): 
            if (X[i,3] >= self.year_ [X[i,0]][X[i,2]][X[i,1]] ): 
              self.year_ [X[i,0]][X[i,2]][X[i,1]]  = X[i,3]
              self.store_[X[i,0]][X[i,2]][X[i,1]] = y[i]
          else:
            self.year_ [X[i,0]][X[i,2]][X[i,1]]  = X[i,3]
            self.store_[X[i,0]][X[i,2]][X[i,1]] = y[i]
        else: 
          self.store_ [X[i,0]][X[i,2]] = {}
          self.year_ [X[i,0]][X[i,2]] = {}
      else: 
        self.store_[X[i,0]] = {}
        self.year_[X[i,0]] = {}
         
    # Return the classifier
    return self

  def predict(self, X):
    # Check is fit had been called
    check_is_fitted(self)
    # Input validation
    X = check_array(X)
    predictions = np.zeros(X.shape[0])
    for i in range(len(predictions)): 
      store = self.store_[X[i,0]]
      month = store[X[i,2]]
      if (X[i,1] in month.keys()):
        predictions[i] = month[X[i,1]]
      elif (X[i,1]-1) in month.keys(): 
        predictions[i] = month[X[i,1]-1]
      else: 
        predictions[i] = month[X[i,1]-2]
    return predictions

In [ ]:
from sklearn.pipeline import  Pipeline 
from sklearn.preprocessing import FunctionTransformer

def Date_speration(X_df): 
    Dates = pd.to_datetime(X_df['Date'])
    Dates = Dates.reset_index(drop= True)
    features = np.zeros(shape=(len(Dates), 3))
    for i in range(len(Dates)): 
      features[i,0] = Dates[i].day
      features[i,1] = Dates[i].month
      features[i,2] = Dates[i].year
    return features
transformer_DtD = FunctionTransformer(
    Date_speration
)

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer

preprocessor_DtD = ColumnTransformer(
    transformers=[
        ('pass', "passthrough", ["Store"]),
        ('date', transformer_DtD, ["Date"])])

In [ ]:
model_DtD = Pipeline(steps=[('preprocessor', preprocessor_DtD),
                      ('model', DateToDateRegressor())])

## Model DayToDay Model

It's bases it's prediction on the day (monday, tuesday,...) and the week (1, .. 53) than the actual date.

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

class DayToDayRegressor(BaseEstimator, RegressorMixin):
  '''
    The model takes a date frame composed of X = [store, dayofWeek, NumberOfWeek, year]
  '''
  def __init__(self, demo_param='demo'):
    self.demo_param = demo_param
  
  def fit(self, X, y):
    # Check that X and y have correct shape
    X, y = check_X_y(X, y)
    # Store the classes seen during fit
    self.year_ =  {}
    self.store_ = {}
    for i in range(len(X)): 
      if (X[i,0] in self.store_.keys()):
        if (X[i,2] in self.store_ [X[i,0]].keys()): 
          if ((X[i,1] in self.year_ [X[i,0]][X[i,2]].keys())): 
            if (X[i,3] >= self.year_ [X[i,0]][X[i,2]][X[i,1]] ): 
              self.year_ [X[i,0]][X[i,2]][X[i,1]]  = X[i,3]
              self.store_[X[i,0]][X[i,2]][X[i,1]] = y[i]
          else:
            self.year_ [X[i,0]][X[i,2]][X[i,1]]  = X[i,3]
            self.store_[X[i,0]][X[i,2]][X[i,1]] = y[i]
        else: 
          self.store_ [X[i,0]][X[i,2]] = {}
          self.year_ [X[i,0]][X[i,2]] = {}
      else: 
        self.store_[X[i,0]] = {}
        self.year_[X[i,0]] = {}
         
    # Return the classifier
    return self

  def predict(self, X):
    # Check is fit had been called
    check_is_fitted(self)
    # Input validation
    X = check_array(X)
    predictions = np.zeros(X.shape[0])
    for i in range(len(predictions)): 
      store = self.store_[X[i,0]]
      if X[i,2] in store.keys():
        key = X[i,2]
      elif (X[i,2] - 1) in store.keys():
        key = X[i,2] - 1
      week = store[key]
      if (X[i,1] in week.keys()):
        predictions[i] = week[X[i,1]]
      elif (X[i,1] == 7):
        predictions[i] = 0 
      elif (X[i,1]-1) in week.keys(): 
        predictions[i] = week[X[i,1]-1]
      else: 
        predictions[i] = week[X[i,1]-2]
    return predictions

In [ ]:
from sklearn.pipeline import  Pipeline 
from sklearn.preprocessing import FunctionTransformer

def  day_of_week(df): 
  Dates = pd.to_datetime(df['Date'])
  Dates = Dates.reset_index(drop= True)
  features = np.zeros(shape=(len(Dates), 3))
  for i in range(len(Dates)):
    x = Dates.iloc[i].isocalendar()
    features[i,0] = x[2]
    features[i,1] = x[1]
    features[i,2] = x[0]
  return features
  
transformer_DytDy = FunctionTransformer(
    day_of_week
)

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer

preprocessor_DytDy = ColumnTransformer(
    transformers=[
        ('pass', "passthrough", ["Store"]),
        ('date', transformer_DytDy, ["Date"])])

In [ ]:
model_DytDy = Pipeline(steps=[('preprocessor', preprocessor_DytDy),
                      ('model', DayToDayRegressor())])

## Model predictive

In [ ]:
from sklearn.pipeline import  Pipeline 
from sklearn.preprocessing import FunctionTransformer
from sklearn.cluster import KMeans

X = df[['Store','Sales']]
kmeans = KMeans(n_clusters = 3)
kmeans.fit(X)
clusters = kmeans.predict(X)
df['shop_cluster'] = kmeans.labels_.astype('int8')

def  dataframe_transformation (df): 
  temp = pd.DataFrame()
  temp["Date"] = pd.to_datetime(df.Date, format = "%Y-%m-%d")
  temp["year"] = pd.DatetimeIndex(temp["Date"]).year
  temp['month'] = pd.DatetimeIndex(temp['Date']).month
  temp['day'] = pd.DatetimeIndex(temp['Date']).day
  temp['quarter'] = pd.DatetimeIndex(temp['Date']).quarter
  temp['season'] = temp.month%12 // 3 + 1
  month_in_year = 12
  temp['month_sin'] = np.sin(2*np.pi*temp.month/month_in_year)
  temp = temp.drop('Date',axis = 1)
  temp['CompetitionYears'] = temp['year'] - df['CompetitionOpenSinceYear']
  temp['CompetitionYears'] = temp['CompetitionYears'].fillna(0)
  temp['CompetitionYears'] = np.clip(temp['CompetitionYears'], a_min= 0, a_max= np.max(temp['CompetitionYears'])+1)
  df = df.join(temp)
  df = df.drop('Date',axis = 1)
  df = df.drop([ 'Customers',
        'Assortment',
        'CompetitionDistance', 'CompetitionOpenSinceMonth',
        'CompetitionOpenSinceYear',  'Promo2SinceWeek',
        'Promo2SinceYear', 'PromoInterval', 'quarter',
        'season'],axis = 1)
  return df

from sklearn.preprocessing import OrdinalEncoder

def labelencoder(df):
  df = df.replace({'StateHoliday':{'0':0,'a':1,'b':2,'c':3}})
  
  df['StateHoliday'] = df['StateHoliday'].fillna(4)
  df["StoreType"] = df["StoreType"].fillna('Unknown')
  
  
  ####
  df["StoreType"] = df["StoreType"].astype('category')
  df["StateHoliday"] = df["StateHoliday"].astype('category')
  
  ###
  df["StoreType"] = OrdinalEncoder().fit_transform(df[['StoreType']])
  df["StateHoliday"] = OrdinalEncoder().fit_transform(df[["StateHoliday"]])
  
  return df
transformer_labels = FunctionTransformer(
    lambda X_df: labelencoder(X_df)
)
othercols = ['Store', 'DayOfWeek', 'Open', 'Promo',
         'shop_cluster',
       'shop_cluster', 'year', 'month', 'day', 'month_sin'
       ,
       'CompetitionYears']
transformer_predictive = make_column_transformer((transformer_labels,['StateHoliday','StoreType']),('passthrough',othercols))

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
othercols = ['Store', 'DayOfWeek', 'Open', 'Promo', 'shop_cluster', "year", "month", "day", "quarter", "season", "month_sin",
             "CompetitionYears"]
preprocessor_predictive = ColumnTransformer(
    transformers=[
        ("label_coder", transformer_labels,['StateHoliday','StoreType']),
        ("pass", 'passthrough', othercols)])

In [ ]:
import xgboost as xg 

model = xg.XGBRegressor(objective='reg:squarederror', 
                  n_estimators= 100, max_depth= 50, gamma= 1, learning_rate= 0.5, n_jobs= -1)

model2 = xg.XGBRegressor(objective='reg:squarederror', 
                  n_estimators= 1000, max_depth= 50, gamma= 1, learning_rate= 0.5, n_jobs= -1)

## Data split and train

In [ ]:
size = int(len(df) /100 * 50) 
df = df.join(df_store, on ="Store")
y = df["Sales"]
X_df = df.drop(columns= "Sales")
X_df = dataframe_transformation(X_df)
X_test = X_df[size:len(df)]
X_train = X_df[3000:size]
y_test = y.iloc[:size]
y_train = y[3000:size]

In [ ]:
y_train1 = y_train.reindex(index= y_train.index[::-1]).values
X_train1 = X_train[0:len(X_train)-1]
X_train1 = X_train1.reindex(index= X_train1.index[::-1])
y_train1 = y_train1[1:len(y_train1)]
y_train1 = np.log1p(y_train1) 
X_train1

,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,shop_cluster,StoreType,Promo2,year,month,day,month_sin,CompetitionYears
508602,948,3,1,1,0,0,0,b,0,2014,4,2,0.866025,0.0
508601,947,3,1,1,0,0,0,a,1,2014,4,2,0.866025,0.0
508600,946,3,1,1,0,0,0,a,1,2014,4,2,0.866025,3.0
508599,945,3,1,1,0,0,0,a,0,2014,4,2,0.866025,3.0
508598,944,3,1,1,0,0,0,c,0,2014,4,2,0.866025,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3004,775,3,1,1,0,0,2,d,1,2015,7,29,-0.500000,10.0
3003,774,3,1,1,0,1,0,a,0,2015,7,29,-0.500000,2.0
3002,773,3,1,1,0,1,0,a,0,2015,7,29,-0.500000,0.0
3001,772,3,1,1,0,1,0,d,0,2015,7,29,-0.500000,0.0


In [ ]:
X_train1 = transformer_predictive.fit_transform(X_train1)

In [ ]:
pip install "statsmodels==0.11.1"

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
x = pd.DataFrame()
x["Dates"] = pd.to_datetime(X_train["Date"])
x["Sales"] = y_train
x = x.reset_index(drop=True)
y_train_arima = x[['Sales','Dates']].groupby(['Dates']).mean().sort_values(by = 'Dates',ascending = True)
model_arima = ARIMA(y_train_arima, order=(1, 2, 3))
model_arima = model_arima.fit()

In [ ]:
import pickle 
filename = 'finalized_model.pickle'

if Model_pre_trained == False: 
  model.fit(X_train1, y_train1)
  f =  open(filename, 'wb')
  pickle.dump(model,f)
  f.close()

elif Model_pre_trained == True:
  f =  open(filename, 'rb')
  model = pickle.load(model,f)
  f.close()

In [ ]:
model.fit(X_train1, y_train1)

In [ ]:
model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=1,
             importance_type='gain', learning_rate=0.5, max_delta_step=0,
             max_depth=50, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=-1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [ ]:
from sklearn.preprocessing import StandardScaler as scaler
import time
def makePredictionBystore(StoreID, weeks, day, month, year, df):
  
  df = df.reindex(index= df.index[::-1])
  df = dataframe_transformation(df)
  df = df[df["Store"] == StoreID]
  df = df.reset_index(drop=True)
  start_idx = df.index[(df['day'] == day) & (df['month'] == month )& (df['year'] == year )& (df["Store"] == StoreID)].to_list()[0]
  X_eval = df.iloc[start_idx+1:len(df),:]
  X = df[0:start_idx+1]
  Data = X[X["Store"] == StoreID].drop('Sales',axis = 1)
  target = X[X["Store"] == StoreID]["Sales"] 
  sales =  X_eval[X_eval["Store"] == StoreID].Sales.values
  Data = transformer_predictive.fit_transform(Data)
  test = X_eval[X_eval["Store"] == StoreID].drop('Sales',axis = 1)
  test = test.reset_index(drop=True)
  
  
  sales = sales[0:7*weeks+1]
  test = test[0:7*weeks+1]

  cl = test['shop_cluster']
  cp = test['StoreType'].values
  
  y_test = (X_eval["Sales"].values)[0:7*weeks+1]
  correction = y_test[0:start_idx+1]
  test = transformer_predictive.fit_transform(test)
  test = scaler().fit_transform(test)
  predictions = []
  target = np.log1p(target)
  sales = np.log1p(sales)
  model2.fit(Data,target)
  
  # A List of Items
  items = list(range(0, 7*weeks+1))
  l = len(items)
  # Initial call to print 0% progress
  printProgressBar(0, l, prefix = 'Progress:', suffix = 'Complete', length = 50)
  for i in range(7*weeks+1):
    t = test[i,:].reshape((1,len(test[i,:])))
    if (cl.values[i]==1) & (cp[i]=='b'):
      p = (0.9*model.predict(t)+model2.predict(t)+1.3*np.mean(target))/2
    else:
      p = (0.9*model.predict(t)+model2.predict(t))/2
    predictions.append(p)
    Data = np.vstack((Data,test[i,:]))
    
    target = np.append(target,p)

    model2.fit(Data,target)
    time.sleep(1)
    printProgressBar(i + 1, l, prefix = 'Progress:', suffix = 'Complete', length = 50)
  return np.expm1(predictions), np.expm1(sales)


In [ ]:
from sklearn.preprocessing import StandardScaler as scaler
import time
def arimaPredictionBystore(StoreID, weeks, day, month, year, df):
  df = df.reindex(index= df.index[::-1])
  df = dataframe_transformation(df)
  df = df[df["Store"] == StoreID]
  df = df.reset_index(drop=True)
  start_idx = df.index[(df['day'] == day) & (df['month'] == month )& (df['year'] == year )& (df["Store"] == StoreID)].to_list()[0]
  X_eval = df.iloc[start_idx+1:len(df),:]
  X = df[0:start_idx+1]
  Data = X[X["Store"] == StoreID].drop('Sales',axis = 1)
  target = X[X["Store"] == StoreID]["Sales"] 
  sales =  X_eval[X_eval["Store"] == StoreID].Sales.values
  Data = transformer_predictive.fit_transform(Data)
  test = X_eval[X_eval["Store"] == StoreID].drop('Sales',axis = 1)
  test = test.reset_index(drop=True)
  
  
  sales = sales[0:7*weeks+1]
  test = test[0:7*weeks+1]
  cl = test['shop_cluster']
  cp = test['StoreType'].values
  
  y_test = (X_eval["Sales"].values)[0:7*weeks+1]
  correction = y_test[0:start_idx+1]
  test = transformer_predictive.fit_transform(test)
  test = scaler().fit_transform(test)
  target = np.log1p(target)
  sales = np.log1p(sales)
  return target, np.expm1(sales), Data, test, cl, cp

In [ ]:
Train_set = pd.read_csv("train.csv")
size = int(len(df) /100 * 50) 
y = Train_set["Sales"]
X_df = Train_set.drop(columns= "Sales")
X_test = Train_set.iloc[:size,:]
X_train = Train_set.iloc[size:,:]
y_test = y.iloc[:size]
y_train = y.iloc[size:]

In [ ]:
def get_data(df = df):
  return Train_set 

In [ ]:
model_DtD.fit(X_train[["Store", "Date"]], y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('pass', 'passthrough',
                                                  ['Store']),
                                                 ('date',
                                                  FunctionTransformer(accept_sparse=False,
                                                                      check_inverse=True,
                                                                      func=<function Date_speration at 0x7f65e43a86a8>,
                                                                      inv_kw_args=None,
                                                                      inverse_func=None,
                                                                      kw_args=None,
      

In [ ]:
model_DytDy.fit(X_train[["Store", "Date"]], y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('pass', 'passthrough',
                                                  ['Store']),
                                                 ('date',
                                                  FunctionTransformer(accept_sparse=False,
                                                                      check_inverse=True,
                                                                      func=<function day_of_week at 0x7f65e3cb8a60>,
                                                                      inv_kw_args=None,
                                                                      inverse_func=None,
                                                                      kw_args=None,
         

In [ ]:
def get_data_pred(df):
  return df = pd.read_csv("train.csv") 

## Interactive widget

In [ ]:

def mape(y_true, y_pred): 
    L=[]
    for i in range(len(y_true)):
      if y_true[i] == 0:
        i = i+1
      else:
        L.append(np.abs((y_true[i] - y_pred[i]) / y_true[i]))
    return np.mean(L) * 100

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm import tqdm

def func(model, day, month, year, store, week, y_fill):
    try: 
      start_date = pd.DataFrame({'year': [year],
                   'month': [month],
                   'day': [day]})
      pd.to_datetime(start_date)
    except ValueError : 
      print("Starting date not acceptable :")
      print(start_date.iloc[0,:])
      return

    number = week*7 + 1
    df = get_data()
    target = pd.DataFrame({'Store': [store for i in range(number)]})
    target["Date"] = pd.date_range(start= str(year) +  "-" + str(month) + "-" + str(day), 
                                     periods= number)

    # Model choice
    if model == "DateToDate": 
      Model = model_DtD
      pred = Model.predict(target)

    elif model == "DayToDay":
      Model = model_DytDy
      pred = Model.predict(target)

    elif model == "Predictive": 
      pred, y_targetpred = makePredictionBystore(StoreID= store, weeks= week,
                                             day= day, month= month, year= year,
                                             df= get_data_pred())
    elif model == "ARIMA": 
      target_arima, y_targetpred, Data, test, cl, cp = arimaPredictionBystore(StoreID= store, weeks= week,
                                             day= day, month= month, year= year,
                                             df= get_data_pred())
      arima_res = np.log1p(model_arima.forecast(steps= number))
      pred = np.zeros(number)
      model2.fit(Data,target_arima-arima_res)
  
      # A List of Items
      items = list(range(0, 7*week+1))
      l = len(items)
      # Initial call to print 0% progress
      pbar = tqdm(total= number)
      for i in range(7*week+1):
        t = test[i,:].reshape((1,len(test[i,:])))
        if (cl.values[i]==1) & (cp[i]=='b'):
          p = (0.9*arima_res[i]+model2.predict(t)+1.3*np.mean(target_arima))/2
        else:
          p = (0.9*arima_res[i]+model2.predict(t))/2
        pred[i] = np.expm1(p)
        Data = np.vstack((Data,test[i,:]))
    
        target_arima = np.append(target_arima,p)

        model2.fit(Data,target_arima)
        time.sleep(1)
        if (week != 0) and (i% week == 0): 
          pbar.update(week)
      pbar.close()
      
    # Acquiring the real target 
    df["Date"] = pd.to_datetime(df["Date"])
    y_target = df.loc[df['Date'].isin(target["Date"]) & (df["Store"] == store)]
    y_target = y_target.reindex(index= y_target.index[::-1])
    y_target = y_target.reset_index(drop=True)
    y_t = np.zeros(len(target))
    j = 0
    index = []
    for i in range(len(target)):
      if len(y_target) == 0: 
        y_t = y_fill + y_t
        break
      if target["Date"][i] == pd.to_datetime(y_target["Date"][j]) : 
        y_t[i] = y_target["Sales"][j]
        index.append(j)
        if j < len(y_target)-1:
          j += 1
      else:
        y_t[i] = y_fill 

    if model == "Predictive" or model == "ARIMA": 
      y_t = y_targetpred
    if week == 0: 
      print("true value: ", y_t)
      print("Predicted sales: ", pred)
      mae_tr = mean_absolute_error(y_t[index], pred[index])
      rmse_tr = mean_squared_error(y_t[index], pred[index], squared= False)
      mape_tr = mape(y_t[index], pred[index])
      print("MAE: ", mae_tr)
      print("RMSE: ", rmse_tr)
      print("MAPE", mape_tr)
      return
    # Plot phase
    plt.close()
    plt.figure(figsize=(20,10))
    plt.plot(target["Date"], pred, label = 'pred')
    plt.plot(target["Date"], y_t, label= 'true')
    plt.xlabel('Days')
    plt.ylabel('Sales')
    plt.xticks(rotation= 45)
    plt.legend()
    if len(index) > 0 : 
      mae_tr = mean_absolute_error(y_t[index], pred[index])
      rmse_tr = mean_squared_error(y_t[index], pred[index], squared= False)
      mape_tr = mape(y_t[index], pred[index])
      
    else : 
      mae_tr = 0
      rmse_tr = 0
      mape_tr = 0
    
    plt.title("The sales of store "+ str(store) + " using model: "+model+" \n starting from " +
              str(day)+"/"+str(month)+"/"+str(year)+ " for "+ str(week)+ " weeks, \n"+ 
              " MAE: "+ str(mae_tr) +", RMSE: "+ str(rmse_tr)+", MAPE: "+ str(mape_tr), fontsize=20)
    plt.grid(True)
    plt.show()
    return  

In [ ]:
inter = interactive(func, day= (1, 31, 1), month= (1, 12, 1),
              year= (2013, 2016, 1), model= ["DateToDate", "DayToDay", "Predictive"],
              store= range(1, 1116), week= (0, 10, 1), y_fill=[-2000, 0, 8000] )

output = inter.children[-1]
output.layout.height = '600px'
output.layout.width = '1000px'

display(inter)

interactive(children=(Dropdown(description='model', options=('DateToDate', 'DayToDay', 'Predictive'), value='D…

In [ ]:
pred

NameError: ignored

In [ ]:
pred,true = makePredictionBystore(11,5,16,6,2014,get_data_pred())

Progress: |██████████████████████████████████████████████████| 100.0% Complete


In [ ]:
pred

array([[0.0268147 ],
       [0.02690261],
       [0.02698679],
       [0.02706963],
       [0.02713747],
       [0.02102929],
       [0.02726021],
       [0.02733373],
       [0.02740614],
       [0.0274753 ],
       [0.02754346],
       [0.02760851],
       [0.02148581],
       [0.0570147 ],
       [0.0572183 ],
       [0.05741908],
       [0.05761294],
       [0.02848922],
       [0.0292395 ],
       [0.02309413],
       [0.0293173 ],
       [0.02936928],
       [0.02941862],
       [0.02946729],
       [0.02951352],
       [0.0295591 ],
       [0.02340358],
       [0.05825186],
       [0.05841535],
       [0.05857677],
       [0.02939169],
       [0.02942631],
       [0.03015864],
       [0.02398942],
       [0.03020129],
       [0.03023368]], dtype=float32)

In [ ]:
y_true, y_pred = arimaPredictionBystore(1,5,16,6,2014,df = get_data_pred())

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(y_true, label = 'pred')
plt.plot(y_pred, label= 'true')
plt.xlabel('Days')
plt.ylabel('Sales')
plt.legend()

In [ ]:
for i in range(1): 
  weeks = 5
  day = 16 
  month = 6
  year = 2014 
  StoreID = 1
  df = get_data_pred()
  df = df.reindex(index= df.index[::-1])
  df = dataframe_transformation(df)
  df = df[df["Store"] == StoreID]
  df = df.reset_index(drop=True)
  print(df[["day","month","year"]])
  start_idx = df.index[(df['day'] == day) & (df['month'] == month )& (df['year'] == year )& (df["Store"] == StoreID)].to_list()[0]
  X_eval = df[start_idx+1:len(df)]
  X = df[0:start_idx+1]
  Data = X[X["Store"] == StoreID].drop('Sales',axis = 1)
  target = X[X["Store"] == StoreID]["Sales"] 
  sales =  X_eval[X_eval["Store"] == StoreID].Sales.values
  Data = transformer_predictive.fit_transform(Data)
  test = X_eval[X_eval["Store"] == StoreID].drop('Sales',axis = 1)
  test = test.reset_index(drop=True)
  
  sales = sales[0:7*weeks+1]
  test = test[0:7*weeks+1]

  cl = test['shop_cluster']
  cp = test['StoreType'].values
  
  y_test = (X_eval["Sales"].values)[0:7*weeks+1]
  correction = y_test[0:start_idx+1]
  test = transformer_predictive.fit_transform(test)
  test = scaler().fit_transform(test)
  predictions = []
  target = np.log1p(target)
  sales = np.log1p(sales)
  model2.fit(Data, target)
  for j in range(2):
    t = test[j,:].reshape((1,len(test[j,:])))
    print("target: ", sales[j])
    print("model 2: ", np.expm1(model2.predict(t)))
    print("model : ", np.expm1(model.predict(t)))
    res = (0.9*model.predict(t)+model2.predict(t))/2
    print(res)
    print(np.expm1(sales[j]))
    print(np.expm1(res))
    print(sales.shape)

In [ ]:
sales[start_idx:start_idx+7*5+1]

In [ ]:
df[(df['day'] == day+1) & (df['month'] == month )& (df['year'] == year )& (df["Store"] == StoreID)]